# Libraries

In [1]:
import  pandas as pd
import  numpy as np
import  plotly.express as px
import  matplotlib.pyplot as plt
import  plotly.graph_objs as go

from    sklearn.feature_selection import RFE
from    sklearn.ensemble import RandomForestClassifier
from    sklearn.impute import SimpleImputer
from    sklearn.preprocessing import StandardScaler
from    imblearn.over_sampling import SMOTE
from    sklearn.feature_selection import SelectFromModel

In [2]:
RANDOM_SEED = 0

In [3]:
cd ..\

c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction


c:\Users\victo\Downloads\00. Estudo Data Science\aps_failure_truck_prediction\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Functions

In [4]:
def get_zero_variance_features(dataframe, target_name: str):
    """
    Receive a dataframe and the target name
    Calculates the variance for each feature
    Return a list with the zero variance features
    """
    columns_to_drop = []
    X = dataframe.drop(columns=target_name)
    for i in X:
        if X[i].std() == 0:
            columns_to_drop.append(i)
            print(f'Column {i} has 0 variance and can be excluded')
    return columns_to_drop

# Train

## Load data

In [10]:
df_air_sys_prev = pd.read_csv('data\\original_data\\air_system_previous_years.csv',
                              encoding='utf-8',
                              na_values=['na']
                              )

# Change columns type to float
columns_object_to_float = df_air_sys_prev.drop(columns='class').columns
df_air_sys_prev[columns_object_to_float] = df_air_sys_prev[columns_object_to_float].astype(float)

# Change class to int dummies
map_class = {'neg': 0, 'pos': 1}
df_air_sys_prev['class'] = df_air_sys_prev['class'].map(map_class)

df_air_sys_prev.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,76698.0,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,0,33058.0,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,0,41040.0,NaN,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,0,12.0,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,0,60874.0,NaN,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [6]:
df_air_sys_prev.dtypes

class       int64
aa_000    float64
ab_000    float64
ac_000    float64
ad_000    float64
           ...   
ee_007    float64
ee_008    float64
ee_009    float64
ef_000    float64
eg_000    float64
Length: 171, dtype: object

In [7]:
df_air_sys_prev.isna().sum()

class         0
aa_000        0
ab_000    46329
ac_000     3335
ad_000    14861
          ...  
ee_007      671
ee_008      671
ee_009      671
ef_000     2724
eg_000     2723
Length: 171, dtype: int64

In [31]:
# # List of columns with NaN values
# columns_with_nan = df_air_sys_prev.isna().sum()[df_air_sys_prev.isna().sum() > 0].index
# columns_with_nan

In [8]:
features = df_air_sys_prev.drop(columns='class').columns
target = 'class'

In [9]:
# Use the median to replace NaN values
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(df_air_sys_prev.drop(columns=target))

SimpleImputer(strategy='median')

In [ ]:
# Replace Nan values of each column with the relative median
df_air_sys_prev = imputer.transform(df_air_sys_prev[features])
df_air_sys_prev.head()

In [ ]:
df_air_sys_prev['class'].value_counts()

In [ ]:
X = df_air_sys_prev.drop(columns=['class'])
y = df_air_sys_prev['class']

# Make the balance of the minority class
oversample = SMOTE(sampling_strategy='minority', random_state=RANDOM_SEED)
X_balanced, y_balanced = oversample.fit_resample(X, y)

# Dataframe with balanced data
df_air_sys_prev_balanced = X_balanced.join(y_balanced)
df_air_sys_prev_balanced.head()

In [ ]:
df_air_sys_prev_balanced['class'].value_counts()

In [ ]:
# Remove zero variance features
columns_to_drop = get_zero_variance_features(df_air_sys_prev_balanced, target_name='class')
df_air_sys_prev_zero_std = df_air_sys_prev_balanced.drop(columns=columns_to_drop).copy()
df_air_sys_prev_zero_std.head()

In [15]:
# # Save train dataframe post-processing
# df_air_sys_prev_zero_std.to_csv('data\processed_data\df_train.csv',
#                                 encoding='utf-8',
#                                 sep=',',
#                                 index=False)

In [20]:
X_train = df_air_sys_prev_zero_std.drop(columns='class')
y_train = df_air_sys_prev_zero_std['class']

In [ ]:
y_train

In [17]:
# Calculate the feature importance from Random Forest
# feature_selector_rand_forest = SelectFromModel(estimator=RandomForestClassifier(random_state=RANDOM_SEED)).fit(X_train, y_train)

In [ ]:
# df_feature_importance = pd.DataFrame({'feature_importance': feature_selector_rand_forest.estimator_.feature_importances_},
#                                       index=feature_selector_rand_forest.estimator_.feature_names_in_
#                                       )
# df_feature_importance.sort_values('feature_importance',
#                                    ascending=False,
#                                    inplace=True
#                                    )
# df_feature_importance

In [22]:
# # Quantile 90%
# q90 = df_feature_importance['feature_importance'].quantile(0.90)

# # Dataframe with values over quantile 90%
# df_feature_importance_over_q90 = df_feature_importance[df_feature_importance['feature_importance'] > q90].copy()
# df_feature_importance_over_q90

In [ ]:
# # Total features over quantile 90%
# df_feature_importance_over_q90.shape[0]

In [23]:
# # Plot features importance
# fig = px.bar(y=df_feature_importance_over_q90['feature_importance'],
#              x=df_feature_importance_over_q90.index,
#              text_auto=True
#              )
# fig.update_traces(textfont=dict(size=14))
# fig.update_layout(height=400,
#                   width=1200,
#                   showlegend=False,
#                   margin=dict(l=10, r=10, b=10, t=10, pad=0),
#                   xaxis=dict(title='Feature', tickfont=dict(size=14)),
#                   yaxis=dict(title='Feature importance',
#                              titlefont=dict(size=14),
#                              dtick=0.01
#                              )
#                   )
# fig.show()

## Graphical analysis

In [24]:
# # Concatenate class with important features
# df_air_sys_prev_important_features = df_air_sys_prev_zero_std[['class'] + df_feature_importance_over_q90.index.to_list()]
# df_air_sys_prev_important_features.head()

In [23]:
# # Calculate and plot features correlations
# corr_air_sys_prev_balanced = df_air_sys_prev_important_features.drop(columns='class').corr()

# fig = px.imshow(corr_air_sys_prev_balanced,
#                 text_auto=True,
#                 color_continuous_scale='Viridis_r',
#                 aspect='auto'
#                 )
# fig.update_layout(height=1000,
#                   width=1000,
#                   showlegend=True,
#                   margin=dict(l=30, r=10, b=10, t=30, pad=0),
#                   yaxis=dict(ticksuffix = "  "),
#                   xaxis=dict(ticksuffix = "  ")
#                   )
# fig.show()

In [24]:
# # Plot the features correlations by class
# fig = px.scatter_matrix(df_air_sys_prev_important_features, color='class',
#                         color_continuous_scale='Viridis_r'
#                         )
# fig.update_traces(diagonal_visible=False)
# fig.update_layout(height=800,
#                   width=800,
#                   showlegend=False,
#                   margin=dict(l=10, r=10, b=10, t=10)
#                   )
# fig.show()

# Test

In [11]:
df_air_sys_present = pd.read_csv('data\\original_data\\air_system_present_year.csv',
                                 encoding='utf-8',
                                 na_values=['na']
                                 )

# Change columns type to float
columns_object_to_float = df_air_sys_present.drop(columns='class').columns
df_air_sys_present[columns_object_to_float] = df_air_sys_present[columns_object_to_float].astype('float64')

# Change class to int dummies
map_class = {'neg': 0, 'pos': 1}
df_air_sys_present['class'] = df_air_sys_present['class'].map(map_class)
df_air_sys_present.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0,60.0,0.0,20.0,12.0,0.0,0.0,0.0,0.0,0.0,...,1098.0,138.0,412.0,654.0,78.0,88.0,0.0,0.0,0.0,0.0
1,0,82.0,0.0,68.0,40.0,0.0,0.0,0.0,0.0,0.0,...,1068.0,276.0,1620.0,116.0,86.0,462.0,0.0,0.0,0.0,0.0
2,0,66002.0,2.0,212.0,112.0,0.0,0.0,0.0,0.0,0.0,...,495076.0,380368.0,440134.0,269556.0,1315022.0,153680.0,516.0,0.0,0.0,0.0
3,0,59816.0,NaN,1010.0,936.0,0.0,0.0,0.0,0.0,0.0,...,540820.0,243270.0,483302.0,485332.0,431376.0,210074.0,281662.0,3232.0,0.0,0.0
4,0,1814.0,NaN,156.0,140.0,0.0,0.0,0.0,0.0,0.0,...,7646.0,4144.0,18466.0,49782.0,3176.0,482.0,76.0,0.0,0.0,0.0


In [12]:
df_air_sys_present.dtypes

class       int64
aa_000    float64
ab_000    float64
ac_000    float64
ad_000    float64
           ...   
ee_007    float64
ee_008    float64
ee_009    float64
ef_000    float64
eg_000    float64
Length: 171, dtype: object

In [13]:
df_air_sys_present.isna().sum()

class         0
aa_000        0
ab_000    12363
ac_000      926
ad_000     3981
          ...  
ee_007      192
ee_008      192
ee_009      192
ef_000      762
eg_000      762
Length: 171, dtype: int64

In [ ]:
df_air_sys_present.isna().sum().sum()